In [ ]:
import json
paper_lists = json.loads(open('dblp_crawler_output.json', 'r').read())

In [ ]:
import pandas as pd
df = pd.DataFrame(paper_lists, columns=["year", "title", "doi", "authors", "ccf_rank", "abbreviation", "ccf_name", "full_name", "publisher"])\
    .sort_values(['ccf_rank', 'year'], ascending=[True, False])
df

In [ ]:
# ------------- Paper Quality -------------
df2 = df[(df['year'] >= 2018) | (df["year"] == -1)]
df2 = df2[(df2['ccf_rank'].str.contains('CCF-A|CCF-B', case=False))]
# ------------- Field         -------------
df2 = df2[(df2['title'].str.contains('reinforcement|multi-armed', case=False))]
# ------------- Direction     -------------
df2 = df2[(df2['title'].str.contains('adversarial', case=False))]
# ------------- Conference / Journal   -------------
df2 = df2[(df2['ccf_name'].str.contains('USENIX|AAAI', case=False))]

df2 = df2.sort_values(['ccf_rank', 'year', 'abbreviation'], ascending=[True, False, False])
df3 = df2[["ccf_rank", "ccf_name", "year", "title", "authors"]].copy() # Show
df3.authors = df3['authors'].str[:32]
print('Count: ', len(df3))
display(df3.head(100).style)
display(pd.DataFrame(df2.groupby(['ccf_rank', 'ccf_name']).size().sort_values(ascending=False)).style)

In [ ]:
# ------------- Conference / Journal   -------------
df2 = df2[(df2['abbreviation'].str.contains('^aaai$', case=False))]

df3 = df2[["ccf_rank", "abbreviation", "year", "title", "authors"]].copy() # Show
df3.authors = df3['authors'].str[:32]
print('Count: ', len(df3))
display(df3.head(100).style)

In [ ]:
open('abstract_crawler_input.json', 'w').write(json.dumps(df2.to_numpy().tolist()))

In [ ]:
target_df = df2.drop(index=[])
["year", "title", "doi", "authors", "ccf_rank", "abbreviation", "ccf_name", "full_name", "publisher"]
output  = '| CCF-rank | Activity Theme | Year | Title |\n'
output += '|---|---|---|---|\n'
for i in range(len(target_df)):
    output += '| %s | %s | %s | [%s](%s) |\n' % (
        target_df.iloc[i]['ccf_rank'], 
        target_df.iloc[i]['ccf_name'], 
        target_df.iloc[i]['year'], 
        target_df.iloc[i]['title'],
        target_df.iloc[i]['doi']
    )
open('table_output.md', 'w').write(output)

In [ ]:
target_df = df2.drop(index=[])
output  = '\n'
for cla in target_df['ccf_rank'].unique():
    class_itme = target_df[target_df['ccf_rank'] == cla]
    output += '## %s (%d)\n' % (cla, len(class_itme))
    for abbr in class_itme['ccf_name'].unique():
        item = class_itme[class_itme['ccf_name'] == abbr]
        output += '### %s (%d)\n' % (abbr, len(item))
        for i in range(len(item)):
            output += '%d. %d, [%s](%s)\n' % (i+1, item.iloc[i]['year'], item.iloc[i]['title'], item.iloc[i]['doi'])
open('list_output.md', 'w').write(output)